In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

# Datum und Uhrzeit für Dateinamen
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Format: YYYY-MM-DD_HH-MM-SS

# TMDb API-Key (ersetzen!)
API_KEY = "1e99de5ed0cf038c6f479de3de969bdf"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# CSV mit TMDb-IDs einlesen
input_file = "../input/tmdb_api/links_1m.csv"  # Datei mit einer Spalte "tmdb_id"
output_file = f"tmdb_filmdaten_{timestamp}.csv"

df = pd.read_csv(input_file)

# Request-Session für bessere Performance
session = requests.Session()

def fetch_movie_data(movie_id):
    """ Holt alle relevanten Filmdaten von TMDb inklusive Cast, Crew & Director """
    try:
        # Filmdetails abrufen mit Credits (Cast & Crew) und Keywords
        details_url = f"{BASE_URL}{movie_id}?api_key={API_KEY}&language=en&append_to_response=keywords,credits"
        response = session.get(details_url)
        
        # Falls die API das Limit erreicht, warten & erneut versuchen
        if response.status_code == 429:
            print(f"Rate Limit erreicht für {movie_id}, warte 5 Sekunden...")
            time.sleep(5)
            return fetch_movie_data(movie_id)  # Erneuter Versuch
        
        details = response.json()

        # Cast & Crew-Daten extrahieren
        cast_list = details.get("credits", {}).get("cast", [])
        crew_list = details.get("credits", {}).get("crew", [])

        # Hauptdarsteller (Top 5)
        top_cast = ", ".join([actor["name"] for actor in cast_list[:5]])

        # Regisseure aus der Crew extrahieren
        directors = ", ".join([member["name"] for member in crew_list if member["job"] == "Director"])

        return {
            "poster_url": f"https://image.tmdb.org/t/p/w500{details.get('poster_path', '')}" if details.get("poster_path") else "",
        }
    except Exception as e:
        print(f"Fehler bei TMDb-ID {movie_id}: {e}")
        return None

# Limitierung auf 50 Requests pro Sekunde
batch_size = 50
movie_data = []
start_time = time.time()

for i in range(0, len(df), batch_size):
    batch = df["id"][i:i+batch_size]  # 50 IDs holen
    print(f"Verarbeite IDs {i+1} bis {i+len(batch)}...")

    batch_results = [fetch_movie_data(mid) for mid in batch]
    movie_data.extend(batch_results)

    # Zwischenspeicherung nach jedem Batch
    pd.DataFrame(movie_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Zwischenspeicherung nach {len(movie_data)} Filmen.")

    # Rate-Limit-Kontrolle
    elapsed_time = time.time() - start_time
    if elapsed_time < 1:
        sleep_time = 1 - elapsed_time
        print(f"Warte {sleep_time:.2f} Sekunden für Rate-Limit...")
        time.sleep(sleep_time)
    
    start_time = time.time()

print(f"Ergebnisse gespeichert in {output_file}")


In [2]:
import os
import requests
import pandas as pd
import time

# TMDb API-Key (ersetzen!)
API_KEY = "1e99de5ed0cf038c6f479de3de969bdf"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# Verzeichnis mit den CSV-Dateien (anpassen!)
input_directory = "./Finished"
output_directory = "./Finished/updated"

# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(output_directory, exist_ok=True)

def fetch_poster_url(movie_id):
    """ Holt die Poster-URL für eine gegebene TMDb-ID """
    try:
        url = f"{BASE_URL}{movie_id}?api_key={API_KEY}&language=de"
        response = requests.get(url).json()
        poster_path = response.get("poster_path", "")
        return f"https://image.tmdb.org/t/p/w500{poster_path}" if poster_path else ""
    except Exception as e:
        print(f"Fehler bei TMDb-ID {movie_id}: {e}")
        return ""

# Alle CSV-Dateien im Verzeichnis durchgehen
for file in os.listdir(input_directory):
    if file.endswith(".csv"):
        input_file_path = os.path.join(input_directory, file)
        output_file_path = os.path.join(output_directory, file.replace(".csv", "_updated.csv"))

        print(f"🔄 Verarbeite Datei: {file} ...")

        # CSV einlesen
        df = pd.read_csv(input_file_path)

        # Prüfen, ob die Spalte "movie_id" existiert
        if "movie_id" not in df.columns:
            print(f"⚠️ Datei {file} enthält keine Spalte 'movie_id'. Überspringe...")
            continue

        # API-Abfrage für jede ID und neue Spalte hinzufügen
        df["poster_url"] = df["movie_id"].apply(lambda x: fetch_poster_url(x) if pd.notna(x) else "")

        # Datei speichern
        df.to_csv(output_file_path, index=False, encoding="utf-8")
        print(f"✅ Gespeichert als: {output_file_path}")

        # TMDb API hat ein Rate Limit → kurze Pause
        print("⏳ Warte 10 Sekunden, um das API-Limit einzuhalten...")
        time.sleep(10)

print("🎉 Alle Dateien erfolgreich verarbeitet!")


🔄 Verarbeite Datei: tmdb_filmdaten_2025-03-01_17-42-18.csv ...


KeyboardInterrupt: 